In [ ]:
import xarray as xr

sagnn = xr.open_dataset('C:/Users/23603526/Documents/GitHub/WeatherEvaluation/data/output_5x7x49x69.nc')


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np

import cartopy.crs as ccrs

from IPython.display import HTML


def plot_example(prediction, seed: int = 0, offset: int = 0, frame_rate: int = 16, levels: int = 10) -> HTML:
    
    lon, lat = prediction.longitude, prediction.latitude
        
    bounds = [lon.min().item(), lon.max().item(), lat.min().item(), lat.max().item()]

    pred = prediction.isel(time=seed)

    init_time_pred = pred.time.values
    
    times_pred = np.array(prediction['prediction_timedelta'].values).astype('timedelta64[ns]')
    times_pred = np.array([np.datetime64(init_time_pred + time) for time in times_pred])
    times = times_pred

    print('Predictions:', times_pred[0])


    fig, axs = plt.subplots(1, 1, figsize=(14, 7), subplot_kw={'projection': ccrs.PlateCarree()})

    vmin = min(pred['wind_speed'])
    vmax = max(pred['wind_speed'])

    fig.subplots_adjust(left=0.05, right=0.95, bottom=0.1, top=0.9, wspace=0.2)

    # print('Predictions:', pred['wind_speed'].shape)

    for ax in axs:
        ax.coastlines()
        ax.set_extent(bounds, crs=ccrs.PlateCarree())
        
    pred_states = pred['wind_speed'].transpose('prediction_timedelta', 'latitude', 'longitude').values

    pr = axs[0].contourf(lon, lat, pred_states[0], levels=levels, vmin=vmin, vmax = vmax, transform=ccrs.PlateCarree())
    axs[0].set_title(f'Predicted {0} - {times[0]}')


    fig.colorbar(pr, ax=axs[0], orientation='vertical', label='Wind Speed (m/s)', shrink=0.3)

    def animate(i):
        for ax in axs:
            ax.clear()
            ax.coastlines()

        axs[0].contourf(lon, lat, pred_states[i], levels=levels, vmin=vmin, vmax = vmax, transform=ccrs.PlateCarree())

        axs[0].set_title(f'Predicted {i} - {times[i]}') # .strftime("%Y-%m-%d %H:%M:%S")
        axs[1].set_title(f'Actual {i} - {times[i]}')
        axs[2].set_title(f'Error {i} - {times[i]}')

            
    frames = pred_states.shape[0]

    interval = 1000 / frame_rate

    ani = FuncAnimation(fig, animate, frames=frames, interval=interval)

    plt.close(fig)

    return HTML(ani.to_jshtml())


plot_example(sagnn, seed=0, offset=1, frame_rate=3, levels=10)

Predictions: 2020-01-01T12:00:00.000000000
Actual: 2020-01-01T12:00:00.000000000


In [10]:
plot_example(era, truth, seed=151, frame_rate=3, levels=10)

Predictions: 2020-03-16T12:00:00.000000000
Actual: 2020-03-16T12:00:00.000000000
